In [1]:
!pip install torch torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Found existing installation: torch-geometric 2.2.0
Uninstalling torch-geometric-2.2.0:
  Successfully uninstalled torch-geometric-2.2.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 18.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/p

In [9]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

1.13.1+cu116


In [10]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

In [11]:
def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()

In [12]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()

In [13]:
print('Dataset properties')
print('==============================================================')
print(f'Dataset: {dataset}') #This prints the name of the dataset
print(f'Number of graphs in the dataset: {len(dataset)}')
print(f'Number of features: {dataset.num_features}') #Number of features each node in the dataset has
print(f'Number of classes: {dataset.num_classes}') #Number of classes that a node can be classified into


#Since we have one graph in the dataset, we will select the graph and explore it's properties

data = dataset[0]
print('Graph properties')
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}') #Number of nodes in the graph
print(f'Number of edges: {data.num_edges}') #Number of edges in the graph
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # Average number of nodes in the graph
print(f'Contains isolated nodes: {data.has_isolated_nodes()}') #Does the graph contains nodes that are not connected
print(f'Contains self-loops: {data.has_self_loops()}') #Does the graph contains nodes that are linked to themselves
print(f'Is undirected: {data.is_undirected()}') #Is the graph an undirected graph

Dataset properties
Dataset: KarateClub()
Number of graphs in the dataset: 1
Number of features: 34
Number of classes: 4
Graph properties
Number of nodes: 34
Number of edges: 156
Average node degree: 4.59
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [ ]:
from torch_geometric.utils import to_networkx

G = to_networkx(data, to_undirected=True)
visualize_graph(G, color=data.y)

In [118]:
import torch
from torch.nn import Linear, GELU
import torch.nn as nn
from torch_geometric.nn.conv.transformer_conv import TransformerConv
from torch.autograd import Variable

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.hidden_size = 4 
        self.i2h = Linear(dataset.num_features + self.hidden_size, self.hidden_size)
        self.i2o = Linear(dataset.num_features + self.hidden_size, 4)
        self.conv = TransformerConv(self.hidden_size + dataset.num_features, 4)
        self.extractor = Linear(4, 4 + self.hidden_size)
        self.mixer = Linear(4 + self.hidden_size, dataset.num_features + self.hidden_size)
        self.classifier = Linear(4, dataset.num_classes)
    def forward(self, input, hidden, edge_index):
        x = torch.cat((input, hidden), 1)
        GELUa = nn.GELU()
        h = self.conv(x, edge_index)
        h = GELUa(h)
        h = self.extractor(h)
        h = self.mixer(h)
        nextHidden = self.i2h(h)
        outputHidden = self.i2o(h)
        out = self.classifier(outputHidden)
        return out, nextHidden
    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

model = GCN()
print(model)

GCN(
  (i2h): Linear(in_features=38, out_features=4, bias=True)
  (i2o): Linear(in_features=38, out_features=4, bias=True)
  (conv): TransformerConv(38, 4, heads=1)
  (extractor): Linear(in_features=4, out_features=8, bias=True)
  (mixer): Linear(in_features=8, out_features=38, bias=True)
  (classifier): Linear(in_features=4, out_features=4, bias=True)
)


In [120]:
model = GCN()
criterion = torch.nn.CrossEntropyLoss()  #Initialize the CrossEntropyLoss function.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Initialize the Adam optimizer.

def train(data):
    optimizer.zero_grad()  # Clear gradients.
    a = torch.zeros(dataset.num_features, 4)
    out, h = model(data.x, a, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

for epoch in range(401):
    loss, h = train(data)
    print(f'Epoch: {epoch}, Loss: {loss}')

Epoch: 0, Loss: 1.4315178394317627
Epoch: 1, Loss: 1.4095698595046997
Epoch: 2, Loss: 1.3964595794677734
Epoch: 3, Loss: 1.3876399993896484
Epoch: 4, Loss: 1.3820247650146484
Epoch: 5, Loss: 1.3792479038238525
Epoch: 6, Loss: 1.3770573139190674
Epoch: 7, Loss: 1.3723692893981934
Epoch: 8, Loss: 1.3639017343521118
Epoch: 9, Loss: 1.351718544960022
Epoch: 10, Loss: 1.3359968662261963
Epoch: 11, Loss: 1.3163366317749023
Epoch: 12, Loss: 1.2915457487106323
Epoch: 13, Loss: 1.2599763870239258
Epoch: 14, Loss: 1.220171570777893
Epoch: 15, Loss: 1.171316146850586
Epoch: 16, Loss: 1.1136327981948853
Epoch: 17, Loss: 1.0488715171813965
Epoch: 18, Loss: 0.9804044365882874
Epoch: 19, Loss: 0.9126631617546082
Epoch: 20, Loss: 0.849560558795929
Epoch: 21, Loss: 0.7926795482635498
Epoch: 22, Loss: 0.7405542135238647
Epoch: 23, Loss: 0.6901300549507141
Epoch: 24, Loss: 0.6420490741729736
Epoch: 25, Loss: 0.6040982604026794
Epoch: 26, Loss: 0.5735111236572266
Epoch: 27, Loss: 0.5345556139945984
Epoch:

In [95]:
torch.cuda.is_available()

True